In [1]:
import pandas as pd
import numpy as np
import os

pd.set_option('display.max_columns', 500)
# Allow pretty-display of multiple variables
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


#
# User-defined CONSTANTS
#
YEAR = 2018
LEAP_YEAR = False # TODO: determine automatically
VERBOSE = False # additional output for debugging and error-checking


#
# Global CONSTANTS
#
if not LEAP_YEAR: 
    HOURS_IN_YEAR = 8760
    DAYS_IN_FEB = 28
else: 
    HOURS_IN_YEAR = 8760 + 24
    DAYS_IN_FEB = 29
    
DAYS_IN_JAN = 31

# DateTimeIndex for resampling time series
START_DATE = str(YEAR) + '-01-01' 
DTI15 = pd.date_range(START_DATE, periods=4*HOURS_IN_YEAR, freq='15min')
DTI30 = pd.date_range(START_DATE, periods=2*HOURS_IN_YEAR, freq='30min')
DTI60 = pd.date_range(START_DATE, periods=HOURS_IN_YEAR, freq='60min')

# DST starts March 26, 2017 / March 25, 2018 / March 31, 2019
DST_START_DAY_MAR = {
    2017 : 26,
    2018 : 25,
    2019 : 31
}

DST_END_DAY_OCT = {
    2017 : 29,
    2018 : 28,
    2019 : 27
}

DST_BOUNDARY_START = pd.Timestamp(year=YEAR, month=3, day=DST_START_DAY_MAR[YEAR], hour=2)
DST_BOUNDARY_END = pd.Timestamp(year=YEAR, month=10, day=DST_END_DAY_OCT[YEAR], hour=2)

EU26 = {'AT', 'BE', 'BG', 'CZ', 'DE', 'DK', 'EE', 'ES', 'FI', 'FR', 'GR', 'HR', 'HU', 
        'IE', 'IT', 'LT', 'LU', 'LV', 'NL', 'PL', 'PT', 'RO', 'SE', 'SI', 'SK', 'UK'}

# Time Intervals: Total Load
LOAD_FREQ = {
    'AT' : 15, 'BE' : 15, 'DE' : 15, 'HU' : 15, 'LU' : 15, 'NL' : 15,
    'IE' : 30, 'UK' : 30, #   'CY' : 30,
    'BG' : 60, 'CZ' : 60, 'DK' : 60, 'EE' : 60, 'ES' : 60, 'FI' : 60,
    'FR' : 60, 'GR' : 60, 'HR' : 60, 'IT' : 60, 'LT' : 60, 'LV' : 60,
    'PL' : 60, 'PT' : 60, 'RO' : 60, 'SE' : 60, 'SI' : 60, 'SK' : 60
}

# Time Intervals: Generation Forecasts
GEN_FORECAST_FREQ = {
    'AT' : 15, 'DE' : 15, 'HU' : 15, 'NL' : 15, #   'CY' : 30,
    'BE' : 60, 'BG' : 60, 'CZ' : 60, 'DK' : 60, 'EE' : 60, 'ES' : 60,
    'FI' : 60, 'FR' : 60, 'HR' : 60, 'IE' : 60, 'IT' : 60, 'LT' : 60,
    'LU' : 60, 'LV' : 60, 'GR' : 60, 'PL' : 60, 'PT' : 60, 'RO' : 60,
    'SE' : 60, 'SI' : 60, 'SK' : 60, 'UK' : 60
}

# Time Intervals: Actual Generation per Production Type
GEN_ACTUAL_FREQ = {
    'AT' : 15, 'DE' : 15, 'HU' : 15, 'NL' : 15, #   'CY' : 30,
    'IE' : 30, 'UK' : 30,
    'BE' : 60, 'BG' : 60, 'CZ' : 60, 'DK' : 60, 'EE' : 60, 'ES' : 60,
    'FI' : 60, 'FR' : 60, 'GR' : 60, 'HR' : 60, 'IT' : 60, 'LT' : 60,
    'LU' : 60, 'LV' : 60, 'PL' : 60, 'PT' : 60, 'RO' : 60, 'SE' : 60,
    'SI' : 60, 'SK' : 60
}

# Time Intervals: Total Imbalance
IMBALANCE_FREQ = {
    'AT' : 15, 'BE' : 15, 'DE' : 15, 'HU' : 15, 'IT' : 15, 'LU' : 15,
    'NL' : 15, 'SK' : 15,
    'FR' : 30, 'IE' : 30, 'UK' : 30,
    'CZ' : 60, 'DK' : 60, 'EE' : 60, 'ES' : 60, 'FI' : 60, 'LV' : 60, 
    'LT' : 60, 'PL' : 60, 'PT' : 60, 'RO' : 60, 'SI' : 60, 'SE' : 60    
}

HAS_NO_BALANCE_DATA = {'BG', 'HR', 'GR'} 
HAS_EXTRA_BALANCE_COL = {'DK', 'FI', 'IE', 'IT', 'SE'}

MV = pd.DataFrame(index=EU26,  
                  columns = ['Max Imbalance','Max Imbalance Hour','Missing'])

HOURS = {} # dictionary of pandas dataframes - hourly data
CRITICAL = {} # dictionary of pandas dataframes - critical hours

In [2]:
#
# TODO: define class
#
        
def load_data_from_csv(country = "DE") :

    # directory where raw csv files reside grouped by year then country
    path = "data" + os.sep + str(YEAR) + os.sep + country + os.sep 
    tail = str(YEAR) + "01010000-" + str(YEAR+1) + "01010000.csv"
    
    vre_forecast_csv_filename = path + "Generation Forecasts for Wind and Solar_" + tail
    print("Loading", vre_forecast_csv_filename)    
    vre_forecast = pd.read_csv(vre_forecast_csv_filename, usecols=[0,1,4,7], na_values=['n/e'])
    vre_forecast.columns = ['Time','Forecast.Solar','Forecast.Offshore','Forecast.Onshore']    
    drop_dst_rows(vre_forecast, GEN_FORECAST_FREQ[country])
    vre_forecast.fillna(0, inplace=True)
    vre_forecast = convert_to_hourly_mean(vre_forecast, GEN_FORECAST_FREQ[country]) 
    if (VERBOSE) : print(vre_forecast)
    
    actual_generation_csv_filename = path + "Actual Generation per Production Type_" + tail
    print("Loading", actual_generation_csv_filename)
    actual_generation = pd.read_csv(actual_generation_csv_filename, usecols=[19,21,22], na_values=['n/e'])
    actual_generation.columns = ['Actual.Solar', 'Actual.Offshore', 'Actual.Onshore']
    drop_dst_rows(actual_generation, GEN_ACTUAL_FREQ[country])
    actual_generation.fillna(0, inplace=True)
    actual_generation = convert_to_hourly_mean(actual_generation, GEN_ACTUAL_FREQ[country])
    if (VERBOSE) : print(actual_generation)
    
    total_load_csv_filename = path + "Total Load - Day Ahead _ Actual_" + tail
    print("Loading", total_load_csv_filename)
    total_load = pd.read_csv(total_load_csv_filename, usecols=[1,2], na_values=['n/e'])
    total_load.columns = ['Forecast.Load', 'Actual.Load']
    drop_dst_rows(total_load, LOAD_FREQ[country])
    total_load = convert_to_hourly_mean(total_load, LOAD_FREQ[country])
    if (VERBOSE) : print(total_load)
        
    if (country in HAS_NO_BALANCE_DATA) : 
        total_load['Imbalance'] = 0.0 # not the cleanest way to go about it, but gets the job done 
        return pd.concat([vre_forecast, actual_generation, total_load], axis=1)
        
    imbalance_csv_filename = path + "Imbalance_" + tail
    print("Loading", imbalance_csv_filename)
    if (country in HAS_EXTRA_BALANCE_COL) : 
        imbalance = pd.read_csv(imbalance_csv_filename, usecols=[4,5], na_values=['n/e'])
    else : 
        imbalance = pd.read_csv(imbalance_csv_filename, usecols=[3,4], na_values=['n/e'])
    imbalance.columns = ['Imbalance','Situation'] # Total Imbalance Volume in MWh
    drop_dst_rows(imbalance, IMBALANCE_FREQ[country])
    df = MV.loc[country]
    # count number of missing values
    df['Missing'] = imbalance['Imbalance'].isna().sum() # / len(imbalance) 
    imbalance.fillna(0, inplace=True)   
    # select rows with "deficit" and change sign
    imbalance.loc[imbalance.Situation == 'Deficit', 'Imbalance'] = -imbalance.Imbalance
    imbalance.drop('Situation', axis=1, inplace=True)
    imbalance = convert_to_hourly_sum(imbalance, IMBALANCE_FREQ[country])
    if (VERBOSE) : print(imbalance)

    return pd.concat([vre_forecast, actual_generation, total_load, imbalance], axis=1)

In [3]:
#
# Handle DST by deleting extra (empty) row(s)
# The DST start date and time is hard-coded to save time
# and because we are only concerned with 2017, 2018, and 2019
#
def drop_dst_rows(df, freq) :

    if (VERBOSE) : print("DST start date is March ", DST_START_DAY_MAR[YEAR])
    start_row = 24 * (DAYS_IN_JAN + DAYS_IN_FEB + (DST_START_DAY_MAR[YEAR] - 1))
    start_row = start_row + 2   # 2am                   
    if (freq == 15) : # 15 minute intervals
        start_row = start_row * 4
        end_row = start_row + 4
    elif (freq == 30) : # 30 minute intervals
        start_row = start_row * 2
        end_row = start_row + 2
    elif (freq == 60) : # hourly intervals
        end_row = start_row + 1
    # else : # raise exception - invalid frequency
    
    if (VERBOSE) : print("DST row(s):", df[start_row:end_row])
    df.drop(df.index[start_row:end_row], inplace=True)    
        
    return


def convert_to_hourly_mean(df, freq) :

    if (freq == 15) : df.index = DTI15   # 15 min intervals        
    elif (freq == 30) : df.index = DTI30 # 30 min intervals
    elif (freq == 60) : df.index = DTI60 # 60 min intervals
    # else : # raise exception - invalid frequency

    return df.resample('H').mean()   


def convert_to_hourly_sum(df, freq) :

    if (freq == 15) : df.index = DTI15   # 15 min intervals        
    elif (freq == 30) : df.index = DTI30 # 30 min intervals
    elif (freq == 60) : df.index = DTI60 # 60 min intervals
    # else : # raise exception - invalid frequency

    return df.resample('H').sum()   


def map_to_raw_timestamp(t) :
        
    T = t.strftime('%d.%m.%Y %H:%M')
    Tplus1 = (t + pd.Timedelta('1 hour')).strftime('%d.%m.%Y %H:%M')
    Tplus2 = (t + pd.Timedelta('2 hours')).strftime('%d.%m.%Y %H:%M')
    
    if (t < DST_BOUNDARY_START) : 
        return (T + " - " + Tplus1)
    elif (t < DST_BOUNDARY_END) : 
        return (Tplus1 + " - " + Tplus2)
    elif (t == DST_BOUNDARY_END) : # Extra added hour from DST transition
        return ("*" + T + " - " + Tplus1 + "*")
    else :
        return (T + " - " + Tplus1)
              

In [4]:
def load_gen_data(country = "DE") :

    # directory where raw csv files reside grouped by year then country
    path = "data" + os.sep + str(YEAR) + os.sep + country + os.sep 
    tail = str(YEAR) + "01010000-" + str(YEAR+1) + "01010000.csv"   
    
    actual_gen_csv_filename = path + "Actual Generation per Production Type_" + tail
    print("Loading", actual_gen_csv_filename)
    
    COLS = [0,1,2,3,4,5,6,7,8,9,10,11,13,14,15,16,17,18,19,20,21,22]
    actual_gen = pd.read_csv(actual_gen_csv_filename, usecols=COLS, na_values=['n/e'])    
    drop_dst_rows(actual_gen, GEN_ACTUAL_FREQ[country])
    actual_gen.fillna(0, inplace=True)     
    actual_gen = convert_to_hourly_mean(actual_gen, GEN_ACTUAL_FREQ[country])

    #actual_gen.columns = ['Biomass','Lignite','Coal Gas','Gas','Coal','Oil','Shale','Peat',
    #                      'Geothermal','Hydro Pumped','Hydro Run','Hydro Water','Hydro Marine',
    #                      'Nuclear','Other','Other RE','Solar','Waste','Offshore','Onshore']
 
    actual_gen.columns = ['Biomass','Fossil Brown Coal','Fossil Coal Gas','Fossil Gas',
                          'Coal','Fossil Oil','Fossil Oil Shale','Fossil Peat',
                          'Geothermal','Hydro Pumped','Hydro Flow','Hydro Water','Hydro Marine',
                          'Nuclear','Other','Other RE','Solar','Waste','Offshore','Onshore']

    # Combine and drop columns    
    # c1) biomass (including entso-e generation type "Biomass")
    # c2) gas (including entso-e generation type "Fossil Coal-derived gas" and "Fossil Gas")
    actual_gen.insert(1, 'Gas', actual_gen['Fossil Coal Gas'] + actual_gen['Fossil Gas'])
    actual_gen.drop(['Fossil Coal Gas','Fossil Gas'], axis=1, inplace=True)
    
    # c3) coal (including entso-e generation type "Fossil Hard coal")    
    # c4) oil (including entso-e generation type "Fossil Oil" and "Fossil Oil shale")
    actual_gen.insert(4, 'Oil', actual_gen['Fossil Oil'] + actual_gen['Fossil Oil Shale'])
    actual_gen.drop(['Fossil Oil','Fossil Oil Shale'], axis=1, inplace=True)
    
    # c5) lignite (including entso-e generation type "Fossil Brown coal/Lignite" and "Fossil Peat")
    actual_gen.insert(5, 'Lignite', actual_gen['Fossil Brown Coal'] + actual_gen['Fossil Peat'])
    actual_gen.drop(['Fossil Brown Coal','Fossil Peat'], axis=1, inplace=True)
    
    # c6) geothermal (including entso-e generation type "Geothermal")
    # c7) hydro storage (including entso-e generation type "Hydro Pumped Storage" BUT ONLY "Actual Aggregated" 
    #     and "Hydro Water Reservoir")
    actual_gen.insert(6, 'Hydro Storage', actual_gen['Hydro Pumped'] + actual_gen['Hydro Water'])
    actual_gen.drop(['Hydro Pumped','Hydro Water'], axis=1, inplace=True)
    
    # c8) hydro flow (including entso-e generation type "Hydro Run-of-river and poundage")    
    # c9) hydro marine (including entso-e generation type "Marine")
    # c10) nuclear (including entso-e generation type "Nuclear")
    # c11) other renewable (including entso-e generation type "Other renewable")
    other = actual_gen['Other']
    actual_gen.drop('Other', axis=1, inplace=True)
    actual_gen['Other'] = other
    
    # c12) solar (including entso-e generation type "Solar")   
    # c13) waste (including entso-e generation type "Waste")
    
    # c14) wind (including entso-e generation type "Wind Offshore" and " Wind Onshore")
    actual_gen.insert(13, 'Wind', actual_gen['Offshore'] + actual_gen['Onshore'])
    actual_gen.drop(['Offshore','Onshore'], axis=1, inplace=True)    
    
    # c15) other non-renewable (including entso-e generation type "Other non-renewable")
    
    if (VERBOSE) : print(actual_gen)

    return actual_gen

In [5]:
def calc_critical_hours(country = "DE") :

    hours = load_data_from_csv(country) 

    # Combine Onshore and Offshore
    hours.insert(4, 'Forecast.Wind', hours['Forecast.Offshore'] + hours['Forecast.Onshore'])
    hours.drop(['Forecast.Offshore','Forecast.Onshore'], axis=1, inplace=True) #hours.head(4)
    hours.insert(3, 'Actual.Wind', hours['Actual.Offshore'] + hours['Actual.Onshore'])
    hours.drop(['Actual.Offshore','Actual.Onshore'], axis=1, inplace=True) #hours.head(4)

    # Calculate VRE
    hours.insert(4, 'Forecast.VRE', hours['Forecast.Solar'] + hours['Forecast.Wind'])
    hours.insert(5, 'Actual.VRE', hours['Actual.Solar'] + hours['Actual.Wind'])

    # Calculate Residual Load and Delta Residual Load
    hours['Forecast.ResLoad'] = hours['Forecast.Load'] - hours['Forecast.VRE']
    hours['Actual.ResLoad'] = hours['Actual.Load'] - hours['Actual.VRE']
    hours['Delta.ResLoad'] = hours['Actual.ResLoad'] - hours['Forecast.ResLoad']
    
    # Add new column with numeric values
    hours['Index'] = np.arange(1, len(hours) + 1) # print(hours.tail(4))
    
    # Append actual generation data
    hours = pd.concat([hours, load_gen_data(country)], axis=1)
    
    # Add table to global dictionary
    HOURS[country] = hours
    
    # Critical hours
    critical_hours = hours.sort_values(by=['Delta.ResLoad'],ascending=False).head(10)
    critical_hours.insert(12, 'Hour', list(critical_hours.index)) # new column of sorted timestamps
    critical_hours['Hour'] = critical_hours['Hour'].apply(map_to_raw_timestamp)

    # Reset index to [1..8760]
    critical_hours.index = list(critical_hours['Index']) 
    critical_hours.drop(columns='Index',inplace=True)
    
    # Move Imbalance column
    imbalance = critical_hours['Imbalance']
    critical_hours.drop('Imbalance', axis=1, inplace=True)
    critical_hours.insert(12, 'Imbalance', imbalance)
    
    # Add table to global dictionary
    CRITICAL[country] = critical_hours
    
    # Export to csv 
    filename = "Critical_Hours_" + country + "_" + str(YEAR) + ".csv" #print(filename)
    critical_hours.to_csv(filename) 
    
    return critical_hours

In [6]:
# Run once to load all data
for country in EU26 : 
    ch = calc_critical_hours(country)
    ch[, ]

Loading data/2018/LU/Generation Forecasts for Wind and Solar_201801010000-201901010000.csv
Loading data/2018/LU/Actual Generation per Production Type_201801010000-201901010000.csv
Loading data/2018/LU/Total Load - Day Ahead _ Actual_201801010000-201901010000.csv
Loading data/2018/LU/Imbalance_201801010000-201901010000.csv
Loading data/2018/LU/Actual Generation per Production Type_201801010000-201901010000.csv


,Forecast.Solar,Actual.Solar,Forecast.Wind,Actual.Wind,Forecast.VRE,Actual.VRE,Forecast.Load,Actual.Load,Forecast.ResLoad,Actual.ResLoad,...,Geothermal,Hydro Storage,Hydro Flow,Hydro Marine,Nuclear,Other RE,Solar,Waste,Wind,Other
776,0.0,0.0,0.0,0.0,0.0,0.0,470.25,730.00,470.25,730.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4081,0.0,0.0,0.0,0.0,0.0,0.0,416.25,639.00,416.25,639.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2264,0.0,0.0,0.0,0.0,0.0,0.0,407.25,624.50,407.25,624.50,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
457,0.0,0.0,0.0,0.0,0.0,0.0,475.75,686.50,475.75,686.50,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2361,0.0,0.0,0.0,0.0,0.0,0.0,481.50,688.75,481.50,688.75,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1869,0.0,0.0,0.0,0.0,0.0,0.0,566.50,772.25,566.50,772.25,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2488,0.0,0.0,0.0,0.0,0.0,0.0,406.25,611.50,406.25,611.50,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
876,0.0,0.0,0.0,0.0,0.0,0.0,614.25,815.75,614.25,815.75,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
981,0.0,0.0,0.0,0.0,0.0,0.0,451.75,650.50,451.75,650.50,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14,0.0,0.0,0.0,0.0,0.0,0.0,305.75,504.50,305.75,504.50,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Loading data/2018/AT/Generation Forecasts for Wind and Solar_201801010000-201901010000.csv
Loading data/2018/AT/Actual Generation per Production Type_201801010000-201901010000.csv
Loading data/2018/AT/Total Load - Day Ahead _ Actual_201801010000-201901010000.csv
Loading data/2018/AT/Imbalance_201801010000-201901010000.csv


/home/igorgierymski/Flex/myprojectenv/lib/python3.5/site-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


Loading data/2018/AT/Actual Generation per Production Type_201801010000-201901010000.csv


...

,Forecast.Solar,Actual.Solar,Forecast.Wind,Actual.Wind,Forecast.VRE,Actual.VRE,Forecast.Load,Actual.Load,Forecast.ResLoad,Actual.ResLoad,...,Geothermal,Hydro Storage,Hydro Flow,Hydro Marine,Nuclear,Other RE,Solar,Waste,Wind,Other
6327,9415.50,7645.75,39557.25,33330.50,48972.75,40976.25,66504.75,71448.75,17532.00,30472.50,...,3.0,543.50,1208.25,0.0,7580.25,129.0,7645.75,735.50,33330.50,182.00
6326,11892.50,10476.75,40030.75,34542.25,51923.25,45019.00,67061.00,73016.75,15137.75,27997.75,...,3.0,424.50,1218.00,0.0,7586.75,129.0,10476.75,745.50,34542.25,182.00
6353,4829.25,3469.50,28669.75,20939.00,33499.00,24408.50,53818.50,57065.50,20319.50,32657.00,...,3.0,1438.00,1274.75,0.0,7731.75,129.0,3469.50,778.50,20939.00,177.00
6325,13899.50,12434.75,39803.25,35991.75,53702.75,48426.50,68100.50,74838.00,14397.75,26411.50,...,3.0,369.75,1241.75,0.0,7587.25,129.0,12434.75,765.25,35991.75,182.00
6328,6640.00,5124.50,38588.00,32582.50,45228.00,37707.00,65207.75,69416.75,19979.75,31709.75,...,3.0,936.50,1202.75,0.0,7636.25,129.0,5124.50,755.25,32582.50,182.00
6352,9159.25,7246.25,30463.75,23572.00,39623.00,30818.25,53668.00,56485.75,14045.00,25667.50,...,3.0,755.75,1285.75,0.0,7562.75,129.0,7246.25,778.25,23572.00,177.00
6259,100.25,80.75,16927.50,15043.75,17027.75,15124.50,63119.75,72342.25,46092.00,57217.75,...,3.0,5089.75,1299.00,0.0,7758.25,129.0,80.75,772.00,15043.75,188.25
6324,14852.25,13574.25,38721.25,35711.25,53573.50,49285.50,69158.50,75900.50,15585.00,26615.00,...,3.0,294.50,1243.50,0.0,7568.50,129.0,13574.25,768.50,35711.25,182.00
6329,3613.75,2634.00,37214.25,31581.00,40828.00,34215.00,64253.25,68651.00,23425.25,34436.00,...,3.0,649.00,1197.50,0.0,7595.00,129.0,2634.00,783.50,31581.00,182.00
8080,551.25,420.25,27375.75,21123.00,27927.00,21543.25,68189.25,72809.75,40262.25,51266.50,...,3.0,2822.75,1259.50,0.0,8873.75,117.0,420.25,786.25,21123.00,397.75


Loading data/2018/ES/Generation Forecasts for Wind and Solar_201801010000-201901010000.csv
Loading data/2018/ES/Actual Generation per Production Type_201801010000-201901010000.csv
Loading data/2018/ES/Total Load - Day Ahead _ Actual_201801010000-201901010000.csv
Loading data/2018/ES/Imbalance_201801010000-201901010000.csv
Loading data/2018/ES/Actual Generation per Production Type_201801010000-201901010000.csv


,Forecast.Solar,Actual.Solar,Forecast.Wind,Actual.Wind,Forecast.VRE,Actual.VRE,Forecast.Load,Actual.Load,Forecast.ResLoad,Actual.ResLoad,...,Geothermal,Hydro Storage,Hydro Flow,Hydro Marine,Nuclear,Other RE,Solar,Waste,Wind,Other
2124,3554.0,2998.0,12970.0,10906.0,16524.0,13904.0,25600.0,26442.0,9076.0,12538.0,...,0.0,3752.0,1669.0,0.0,6073.0,96.0,2998.0,261.0,10906.0,13.0
1665,493.0,387.0,14450.0,11677.0,14943.0,12064.0,22522.0,22908.0,7579.0,10844.0,...,0.0,3397.0,1755.0,0.0,5055.0,78.0,387.0,272.0,11677.0,53.0
6877,2484.0,1967.0,10359.0,8157.0,12843.0,10124.0,25509.0,26048.0,12666.0,15924.0,...,0.0,1054.0,676.0,0.0,6991.0,98.0,1967.0,311.0,8157.0,58.0
2128,3277.0,2800.0,14315.0,12021.0,17592.0,14821.0,23769.0,24102.0,6177.0,9281.0,...,0.0,3076.0,1559.0,0.0,6073.0,96.0,2800.0,258.0,12021.0,13.0
2139,97.0,69.0,13511.0,10665.0,13608.0,10734.0,20653.0,20881.0,7045.0,10147.0,...,0.0,3056.0,1514.0,0.0,6076.0,89.0,69.0,259.0,10665.0,13.0
2129,2718.0,2308.0,14385.0,12210.0,17103.0,14518.0,23411.0,23836.0,6308.0,9318.0,...,0.0,3266.0,1583.0,0.0,6072.0,95.0,2308.0,254.0,12210.0,13.0
6878,2452.0,1915.0,9952.0,7788.0,12404.0,9703.0,25307.0,25591.0,12903.0,15888.0,...,0.0,709.0,660.0,0.0,6993.0,100.0,1915.0,312.0,7788.0,57.0
2141,53.0,69.0,13385.0,10495.0,13438.0,10564.0,20446.0,20552.0,7008.0,9988.0,...,0.0,3026.0,1508.0,0.0,6077.0,90.0,69.0,264.0,10495.0,13.0
1408,767.0,677.0,10320.0,9010.0,11087.0,9687.0,34692.0,36229.0,23605.0,26542.0,...,0.0,5086.0,1301.0,0.0,7099.0,103.0,677.0,306.0,9010.0,53.0
1407,922.0,755.0,10035.0,8597.0,10957.0,9352.0,35422.0,36696.0,24465.0,27344.0,...,0.0,6046.0,1276.0,0.0,7099.0,103.0,755.0,298.0,8597.0,52.0


Loading data/2018/PL/Generation Forecasts for Wind and Solar_201801010000-201901010000.csv
Loading data/2018/PL/Actual Generation per Production Type_201801010000-201901010000.csv
Loading data/2018/PL/Total Load - Day Ahead _ Actual_201801010000-201901010000.csv
Loading data/2018/PL/Imbalance_201801010000-201901010000.csv
Loading data/2018/PL/Actual Generation per Production Type_201801010000-201901010000.csv


,Forecast.Solar,Actual.Solar,Forecast.Wind,Actual.Wind,Forecast.VRE,Actual.VRE,Forecast.Load,Actual.Load,Forecast.ResLoad,Actual.ResLoad,...,Geothermal,Hydro Storage,Hydro Flow,Hydro Marine,Nuclear,Other RE,Solar,Waste,Wind,Other
3636,0.0,0.0,200.0,214.0,200.0,214.0,20200.0,21275.0,20000.0,21061.0,...,0.0,189.0,139.0,0.0,0.0,0.0,0.0,0.0,214.0,0.0
7861,0.0,0.0,204.0,210.0,204.0,210.0,21600.0,22660.0,21396.0,22450.0,...,0.0,17.0,80.0,0.0,0.0,0.0,0.0,0.0,210.0,0.0
7828,0.0,0.0,331.0,162.0,331.0,162.0,17200.0,18049.0,16869.0,17887.0,...,0.0,0.0,104.0,0.0,0.0,0.0,0.0,0.0,162.0,0.0
7860,0.0,0.0,234.0,238.0,234.0,238.0,21900.0,22888.0,21666.0,22650.0,...,0.0,32.0,80.0,0.0,0.0,0.0,0.0,0.0,238.0,0.0
7827,0.0,0.0,307.0,154.0,307.0,154.0,17200.0,18001.0,16893.0,17847.0,...,0.0,0.0,104.0,0.0,0.0,0.0,0.0,0.0,154.0,0.0
7819,0.0,0.0,506.0,212.0,506.0,212.0,24900.0,25540.0,24394.0,25328.0,...,0.0,64.0,173.0,0.0,0.0,0.0,0.0,0.0,212.0,0.0
8064,0.0,0.0,3489.0,2801.0,3489.0,2801.0,17300.0,17546.0,13811.0,14745.0,...,0.0,0.0,77.0,0.0,0.0,0.0,0.0,0.0,2801.0,0.0
7863,0.0,0.0,210.0,200.0,210.0,200.0,20900.0,21823.0,20690.0,21623.0,...,0.0,1.0,125.0,0.0,0.0,0.0,0.0,0.0,200.0,0.0
3635,0.0,0.0,151.0,128.0,151.0,128.0,20100.0,21007.0,19949.0,20879.0,...,0.0,168.0,139.0,0.0,0.0,0.0,0.0,0.0,128.0,0.0
8063,0.0,0.0,3544.0,2859.0,3544.0,2859.0,18400.0,18632.0,14856.0,15773.0,...,0.0,0.0,77.0,0.0,0.0,0.0,0.0,0.0,2859.0,0.0


In [ ]:
# Find Max Imbalance values
for index, row in MV.iterrows():
    country = index
    imbal_col = HOURS[country]['Imbalance'] # select column
    #max_value = max(abs(imbal_col))
    idx = imbal_col.idxmax()
    row[0] = imbal_col[idx]
    row[1] = map_to_raw_timestamp(idx)
    #row[2] = imbal_col[idx]

MV.sort_index(inplace=True)
MV

# Export to csv 
filename = "Max_Values_" + str(YEAR) + ".csv" #print(filename)
MV.to_csv(filename) 

In [ ]:
# TEST functions

test = load_data_from_csv("SE")
test.head(24)

#for country in LOAD_FREQ : # iterate thru keys in dictionary (in random order)
#    test = load_gen_data(country)
#    test

test = load_gen_data('DE')
test
test.iloc[:,0:11]

#for country in LOAD_FREQ : # iterate thru keys in dictionary (in random order)
#    gen = load_gen_data(country)
#    gen

#actual_gen = load_gen_data('DE')
#actual_gen
#actual_gen.iloc[:,0:11]

index = 7200    # Test cases: 1990, 7200, 8754 
#for i in range(index, index+6): print(map_to_raw_timestamp(DTI60[i]))
